<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [3]:
p=che.Props(['Benzene','Toluene'])

In [10]:
d=DotDict()

d.P = 150e3
d.N = 4
d.NC = p.Name.size
d.F= 100.
d.Fz = np.array([0.5,0.5])
d.FT = 370.
d.D = d.F/2
d.B = d.F - d.D

d.F_stage = 2


d.U = np.zeros(d.N)
d.U[0]=d.D
d.U[-1]=d.B

d.reflux_ratio = 3.

d.T = RangeArray([377.46245809, 382.48934105, 387.71326265], 367., 398.)
d.TC = Range(370., 360.,400.)

d.L = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N-1), 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N-1), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )

d.Lx = CompArray(np.tile(d.Fz,(3,1)))
d.Vy = CompArray(np.tile(d.Fz,(3,1)))


# d.Lx = CompArray(np.array([[0.55902692, 0.44097308],
#         [0.39606249, 0.60393751],
#         [0.24765837, 0.75234163]]))
# d.Vy =  CompArray(np.array([[0.75234163, 0.24765837],
#         [0.6073556 , 0.3926444 ],
#         [0.43316352, 0.56683648]]))



In [11]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [15]:
r = DotDict()

r.MB = np.zeros((d.N-1,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]*p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = np.array(p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:]))
    r.EB[d.F_stage-1] += np.array(p.Hl(d.F*d.Fz, d.FT))
    r.EB[-1]+=d.QB

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF]), r


In [16]:
eqs_wrapped = wrap(eqs)

In [17]:
eqs(d2)

(array([ 2.59354281e+04, -3.28786572e+04,  4.00113196e+04, -2.62394417e+04,
         5.61777337e+04, -1.84927773e+04,  0.00000000e+00,  0.00000000e+00,
         5.00000000e+01,  5.00000000e+01, -2.50000000e+01, -2.50000000e+01,
        -3.77089072e+05,  3.93023844e+06, -6.99402138e+06, -3.38736913e+04,
         2.00000000e+00]),
 {'EB': array([ -377089.07171622,  3930238.43782895, -6994021.37552386]),
  'EQUIL': DeviceArray([[ 25935.42805913, -32878.65724718],
               [ 40011.31961958, -26239.44171565],
               [ 56177.73370987, -18492.7773435 ]], dtype=float64),
  'Lin': array([[150., 150.],
         [150., 150.],
         [150., 150.]]),
  'Lout': array([[150., 150.],
         [150., 150.],
         [ 25.,  25.]]),
  'MB': array([[  0.,   0.],
         [ 50.,  50.],
         [-25., -25.]]),
  'RF': array([2.]),
  'T': array([370.        , 377.46245809, 382.48934105, 387.71326265]),
  'TCBP': array([-33873.69127785]),
  'Vin': array([[150., 150.],
         [150., 150.],


In [ ]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

In [19]:
dunk=xtodunk(solx)
dunk

{'L': array([150.00000004, 149.98397176, 255.83763635]),
 'Lx': array([[0.55946779, 0.44053221],
        [0.39647357, 0.60352643],
        [0.2473177 , 0.7526823 ]]),
 'QB': array(6659741.07083134),
 'T': array([377.44973286, 382.47578968, 387.72607077]),
 'TC': array(372.26289505),
 'V': array([200.        , 199.98397175, 205.83763634]),
 'Vy': array([[0.7526823 , 0.2473177 ],
        [0.60777529, 0.39222471],
        [0.43270501, 0.56729499]])}

In [20]:
eqs(xtod(sol.x,d2))

(array([-3.93396331e-05,  4.04310485e-05, -5.75344893e-06,  4.31279477e-06,
        -1.96101762e-04,  8.10245692e-05,  4.03670981e-08, -1.85665314e-08,
        -1.20174846e-07,  1.36761727e-07,  1.53869834e-07, -1.52084695e-07,
         2.34611705e-03, -3.19478335e-03,  4.51472774e-03,  2.42040551e-05,
         2.48467913e-12]),
 {'EB': array([ 0.00234612, -0.00319478,  0.00451473]),
  'EQUIL': DeviceArray([[-3.93396331e-05,  4.04310485e-05],
               [-5.75344893e-06,  4.31279477e-06],
               [-1.96101762e-04,  8.10245692e-05]], dtype=float64),
  'Lin': array([[112.90234552,  37.09765452],
         [ 83.91120183,  66.07276994],
         [101.43286048, 154.40477587]]),
  'Lout': array([[ 83.91120183,  66.07276994],
         [101.43286048, 154.40477587],
         [ 12.36588479,  37.63411521]]),
  'MB': array([[ 4.03670981e-08, -1.85665314e-08],
         [-1.20174846e-07,  1.36761727e-07],
         [ 1.53869834e-07, -1.52084695e-07]]),
  'RF': array([2.48467913e-12]),
  'T'